<a href="https://colab.research.google.com/github/sejongsmarcle/2023_Spring_Kaggle_Study/blob/main/%EC%8A%A4%ED%84%B0%EB%94%94%20%EC%8B%9C%EA%B0%84/3week/9%ED%8C%80/%ED%96%A5%EC%9E%84_%EC%A7%80%EC%9C%A4_3%ED%9A%8C%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import lightgbm as lgb
import folium
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
submission = pd.read_csv('/content/submission_제출양식.csv')

In [ ]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  object 
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 22.5+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  object 
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(3), object(5)
memory usage: 9.1+ MB


In [ ]:
train.describe()

,id,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude,next_arrive_time
count,210457.000000,2.104570e+05,2.104570e+05,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000
mean,105228.000000,4.052491e+08,7.988694e+06,33.434528,126.603451,490.256100,33.434711,126.603687,85.380824
std,60753.847139,9.132404e+04,6.774077e+03,0.102350,0.123961,520.563932,0.102224,0.123838,85.051170
min,0.000000,4.051360e+08,7.983000e+06,33.244382,126.473300,97.000000,33.244382,126.473300,6.000000
25%,52614.000000,4.051365e+08,7.983093e+06,33.325283,126.523900,291.000000,33.325283,126.524550,44.000000
50%,105228.000000,4.053201e+08,7.983431e+06,33.484667,126.551050,384.000000,33.484860,126.551050,66.000000
75%,157842.000000,4.053201e+08,7.997041e+06,33.500197,126.650322,542.000000,33.500228,126.650322,102.000000
max,210456.000000,4.053281e+08,7.997124e+06,33.556167,126.935188,7461.000000,33.556167,126.935188,2996.000000


In [ ]:
train[['now_arrive_time','next_arrive_time']]

,now_arrive_time,next_arrive_time
0,06시,24
1,06시,36
2,06시,40
3,06시,42
4,07시,64
...,...,...
210452,21시,96
210453,21시,50
210454,21시,16
210455,21시,38


In [ ]:
train

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210452,210452,2019-10-28,405328102,7983486,281-2,33.255783,126.577450,비석거리,21시,528.0,삼아아파트,33.251896,126.574417,96
210453,210453,2019-10-28,405328102,7983486,281-2,33.248595,126.568527,동문로터리,21시,280.0,매일올레시장 7번입구,33.249753,126.565959,50
210454,210454,2019-10-28,405328102,7983486,281-2,33.251891,126.560303,서귀포시 구 버스터미널,21시,114.0,아랑조을거리 입구,33.251084,126.559551,16
210455,210455,2019-10-28,405328102,7983486,281-2,33.251084,126.559551,아랑조을거리 입구,21시,223.0,평생학습관,33.249504,126.558068,38


In [ ]:
train['now_arrive_time']

0         06시
1         06시
2         06시
3         06시
4         07시
         ... 
210452    21시
210453    21시
210454    21시
210455    21시
210456    21시
Name: now_arrive_time, Length: 210457, dtype: object

In [ ]:
train['now_arrive_time'].str.slice(stop=-1).astype(int)

0          6
1          6
2          6
3          6
4          7
          ..
210452    21
210453    21
210454    21
210455    21
210456    21
Name: now_arrive_time, Length: 210457, dtype: int64

In [ ]:
train['now_arrive_time'] = train['now_arrive_time'].str.slice(stop=-1).astype(int)

In [ ]:
test['now_arrive_time'] = test['now_arrive_time'].str.slice(stop=-1).astype(int)

In [ ]:
test

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,7,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,7,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,8,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,8,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,8,550.0,케이티앤지,33.482077,126.485355
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91769,302350,2019-11-05,405328102,7983486,281-2,33.493625,126.534764,제주지방법원(광양방면),20,272.0,고산동산(광양방면),33.495540,126.532907
91770,302351,2019-11-05,405328102,7983486,281-2,33.495540,126.532907,고산동산(광양방면),20,447.0,제주시청(광양방면),33.498925,126.530351
91771,302352,2019-11-05,405328102,7983486,281-2,33.498925,126.530351,제주시청(광양방면),20,418.0,광양사거리,33.500473,126.527103
91772,302353,2019-11-05,405328102,7983486,281-2,33.500473,126.527103,광양사거리,20,140.0,탐라장애인 종합복지관,33.500228,126.525625


In [ ]:
train

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,6,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,6,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,6,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),6,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,7,374.0,은남동,33.485822,126.490897,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210452,210452,2019-10-28,405328102,7983486,281-2,33.255783,126.577450,비석거리,21,528.0,삼아아파트,33.251896,126.574417,96
210453,210453,2019-10-28,405328102,7983486,281-2,33.248595,126.568527,동문로터리,21,280.0,매일올레시장 7번입구,33.249753,126.565959,50
210454,210454,2019-10-28,405328102,7983486,281-2,33.251891,126.560303,서귀포시 구 버스터미널,21,114.0,아랑조을거리 입구,33.251084,126.559551,16
210455,210455,2019-10-28,405328102,7983486,281-2,33.251084,126.559551,아랑조을거리 입구,21,223.0,평생학습관,33.249504,126.558068,38


In [ ]:
train['now_station'].value_counts()

남국원(아라방면)     3361
아라주공아파트       3098
도호동           3093
인다마을          3056
은남동           2789
              ... 
상도리 세화고등학교      25
오조리상동입구         21
세화고등학교          18
하도리 개긋물         11
전경대 입구           7
Name: now_station, Length: 348, dtype: int64

In [ ]:
train['now_station'].value_counts()

남국원(아라방면)     3361
아라주공아파트       3098
도호동           3093
인다마을          3056
은남동           2789
              ... 
상도리 세화고등학교      25
오조리상동입구         21
세화고등학교          18
하도리 개긋물         11
전경대 입구           7
Name: now_station, Length: 348, dtype: int64

In [ ]:
train['next_station'].value_counts()

은남동                3519
제주여자중고등학교(아라방면)    3207
아라주공아파트            3190
인다마을               3105
제원아파트              2553
                   ... 
서귀포동부도서관             12
하도리 개긋물              10
전경대 입구                7
제주버스터미널(가상정류소)        3
제주한라대학교               2
Name: next_station, Length: 351, dtype: int64

In [ ]:
train['next_station'].value_counts()

은남동                3519
제주여자중고등학교(아라방면)    3207
아라주공아파트            3190
인다마을               3105
제원아파트              2553
                   ... 
서귀포동부도서관             12
하도리 개긋물              10
전경대 입구                7
제주버스터미널(가상정류소)        3
제주한라대학교               2
Name: next_station, Length: 351, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit(train['next_station'])

train['now_station'] = le.transform(train['now_station']) 
test['now_station'] = le.transform(test['now_station'])  

train['next_station'] = le.transform(train['next_station'])  
test['next_station'] = le.transform(test['next_station'])  

In [ ]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,239,6,266.0,227,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,227,6,333.0,228,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,228,6,415.0,235,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,41,6,578.0,249,33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,63,7,374.0,221,33.485822,126.490897,64


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  int64  
 8   now_arrive_time   210457 non-null  int64  
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  int64  
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(7), object(2)
memory usage: 22.5+ MB


In [ ]:
print(train['date'].max())
print(train['date'].min())

print(test['date'].max())
print(test['date'].min())

2019-10-28
2019-10-15
2019-11-05
2019-10-29


In [ ]:
train['route_nm'].unique()

array(['360-1', '360-2', '360-7', '360-12', '365-21', '365-22', '201-11',
       '201-12', '201-13', '201-14', '201-15', '201-16', '201-17',
       '201-18', '201-21', '201-22', '201-24', '201-26', '201-27',
       '281-1', '281-2'], dtype=object)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  int64  
 8   now_arrive_time   210457 non-null  int64  
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  int64  
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(7), object(2)
memory usage: 22.5+ MB


In [ ]:
feature =[
    'now_latitude','now_longitude','now_station','now_arrive_time',
    'distance','next_station','next_latitude','next_longitude'
]

x_train = train[feature]
y_train = train['next_arrive_time']
x_test = test[feature]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from pandas.core.common import random_state


model_dict = {
    """
    'rf1': RandomForestRegressor(n_estimators = 100, random_state = 0), 
    'rf2': RandomForestRegressor(n_estimators = 200, random_state = 0), 
    'rf3': RandomForestRegressor(n_estimators = 300, random_state = 0), 
    'lgb1': lgb.LGBMRegressor(random_state = 0),
    'lgb2': lgb.LGBMRegressor(random_state = 0, num_leaves = 41),
    'lgb3': lgb.LGBMRegressor(random_state = 0, num_leaves = 51, min_data_in_leaf = 30)

    """

    'lgb3-1':lgb.LGBMRegressor(random_state = 0, num_leaves = 51),
    'lgb4':lgb.LGBMRegressor(random_state = 0, num_leaves = 61)
}

In [ ]:
model_dict.keys()

dict_keys(["\n    'rf1': RandomForestRegressor(n_estimators = 100, random_state = 0), \n    'rf2': RandomForestRegressor(n_estimators = 200, random_state = 0), \n    'rf3': RandomForestRegressor(n_estimators = 300, random_state = 0), \n    'lgb1': lgb.LGBMRegressor(random_state = 0),\n    'lgb2': lgb.LGBMRegressor(random_state = 0, num_leaves = 41),\n    'lgb3': lgb.LGBMRegressor(random_state = 0, num_leaves = 51, min_data_in_leaf = 30)\n\n    lgb3-1", 'lgb4'])

In [ ]:
model_result={}

for key in model_dict.keys():
  print("#### 훈련중 ####")
  model_dict[key].fit(x_train,y_train)
  print("#### 예측중 ####")
  model_result[key] = model_dict[key].predict(x_test)
  

#### 훈련중 ####
#### 예측중 ####
#### 훈련중 ####
#### 예측중 ####


In [ ]:
model_result

{"\n    'rf1': RandomForestRegressor(n_estimators = 100, random_state = 0), \n    'rf2': RandomForestRegressor(n_estimators = 200, random_state = 0), \n    'rf3': RandomForestRegressor(n_estimators = 300, random_state = 0), \n    'lgb1': lgb.LGBMRegressor(random_state = 0),\n    'lgb2': lgb.LGBMRegressor(random_state = 0, num_leaves = 41),\n    'lgb3': lgb.LGBMRegressor(random_state = 0, num_leaves = 51, min_data_in_leaf = 30)\n\n    lgb3-1": array([ 41.3931752 ,  55.41031328, 123.91431217, ..., 111.88744607,
         32.36994505,  97.83478447]),
 'lgb4': array([ 42.18249926,  57.41674861, 130.15185933, ..., 112.03845332,
         34.0541938 ,  96.93258601])}

In [ ]:
"""
rf1_submit=submission.copy()
rf2_submit=submission.copy()
rf3_submit=submission.copy()

lgb1_submit=submission.copy()
lgb2_submit=submission.copy()
lgb3_submit=submission.copy()

"""

lgb3-1_submit=submission.copy()
lgb4_submit=submission.copy()

SyntaxError: ignored

In [ ]:
"""
rf1_submit['next_arrive_time'] = model_result['rf1']
rf2_submit['next_arrive_time'] = model_result['rf2']
rf3_submit['next_arrive_time'] = model_result['rf3']

lgb1_submit['next_arrive_time'] = model_result['lgb1']
lgb2_submit['next_arrive_time'] = model_result['lgb2']
lgb3_submit['next_arrive_time'] = model_result['lgb3']

"""
lgb3-1_submit['next_arrive_time'] = model_result['lgb3-1']
lgb4_submit['next_arrive_time'] = model_result['lgb4']

In [ ]:
lgb3-1_submit.head()

In [ ]:

"""
rf1_submit.to_csv('rf1_submit.csv',index = False)
rf2_submit.to_csv('rf2_submit.csv',index = False)
rf3_submit.to_csv('rf3_submit.csv',index = False)

lgb1_submit.to_csv('lgb1_submit.csv',index = False)
lgb2_submit.to_csv('lgb2_submit.csv',index = False)
lgb3_submit.to_csv('lgb3_submit.csv',index = False)
"""

lgb3-1_submit.to_csv('lgb3-1_submit.csv',index = False)
lgb4_submit.to_csv('lgb4_submit.csv',index = False)